In [1]:
import json
import time
from pathlib import Path
import numpy as np

from rag.reranker import CrossEncoderReranker
from rag.rerankers.reranker_bge import BGEReranker
from rag.rerankers.reranker_cohere import CohereReranker

TOP_K = 5

def evaluate(reranker_name, reranker, retrieval_data):
    mrr_scores = []
    hits = 0
    latencies = []

    for i, item in enumerate(retrieval_data):
        final_query = item["rewritten_query"]
        answer_span = item["answer_span"]
        chunks = [c["content"] for c in item["chunks"]]

        start = time.perf_counter()

        reranked = reranker.rerank(final_query, chunks, top_k=TOP_K)

        rank = None
        for idx, c in enumerate(reranked):
            if answer_span.lower() in c["content"].lower():
                rank = idx + 1
                break

        if rank:
            hits += 1
            mrr_scores.append(1 / rank)
        else:
            mrr_scores.append(0)

        latencies.append(time.perf_counter() - start)

    return {
        "MRR": float(np.mean(mrr_scores)),
        "Recall@5": hits / len(retrieval_data),
        "latency_ms": {
            "mean": float(np.mean(latencies) * 1000),
            "p95": float(np.percentile(latencies, 95) * 1000),
        },
        "reranker": reranker_name,
    }

def run():
    # Load pre-retrieved chunks
    retrieval_data = json.loads(Path("retrieved_chunks.json").read_text())

    experiments = {
        "cross_encoder_minilm": CrossEncoderReranker(),
        "bge_reranker_base": BGEReranker(),
        "cohere_rerank": CohereReranker("YOUR_API_KEY"),
    }

    results = {}

    for name, reranker in experiments.items():
        print("Running:", name)
        results[name] = evaluate(name, reranker, retrieval_data)

    Path("reranker_ablation_results.json").write_text(
        json.dumps(results, indent=2)
    )

if __name__ == "__main__":
    run()

ModuleNotFoundError: No module named 'rag'